In [1]:
from typing import List, OrderedDict
import os

import flwr
import numpy as np
import torch
from ultralytics import YOLO

/home/pierre/venvs/flwr/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-16 18:19:36,814	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
def set_parameters(model, parameters: List[np.ndarray]):
    params_dict = zip(model.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    model.load_state_dict(state_dict, strict=True)

In [3]:
def get_parameters(model) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in model.state_dict().items()]

In [4]:
class YOLOClient(flwr.client.NumPyClient):
    def __init__(self, model, dataset):
        self.model = model
        self.config_path = f"datasets/{dataset}/config.yaml"
        self.train_size = len(os.listdir(f"datasets/{dataset}/images/train"))

    def get_parameters(self, config):
        return get_parameters(self.model)

    def fit(self, parameters, config):
        set_parameters(self.model, parameters)
        self.model.train(data=self.config_path, epochs=1)
        return get_parameters(self.model), self.train_size, {}

    def evaluate(self, parameters, config):
        set_parameters(self.model, parameters)
        metrics = self.model.val()
        loss = metrics.box.loss
        accuracy = metrics.results_dict.get("metrics/precision(B)")
        return loss, self.train_size, {"accuracy": accuracy}

In [5]:
datasets = ["roadsigns", "citypersons"]

def client_fn(cid: str) -> YOLOClient:
    model = YOLO(model="runs/detect/train5/weights/best.pt")
    dataset = datasets[int(cid)]
    return YOLOClient(model, dataset).to_client()

In [6]:
NUM_CLIENTS = 2
BATCH_SIZE = 32

In [7]:
strategy = flwr.server.strategy.FedAvg()
client_resources = {"num_cpus": 1, "num_gpus": 1.0}

flwr.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=flwr.server.ServerConfig(num_rounds=1),
    strategy=strategy,
    client_resources=client_resources,
)

INFO :      Starting Flower simulation, config: num_rounds=1, no round_timeout
2024-06-16 18:19:42,718	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'object_store_memory': 4053002649.0, 'accelerator_type:G': 1.0, 'GPU': 1.0, 'memory': 8106005300.0, 'node:192.168.2.120': 1.0, 'CPU': 12.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 1.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
INFO :      Received initial parameters from one random client
INFO :      Evaluating initial global parameters
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)


(ClientAppActor pid=6270) Ultralytics YOLOv8.2.32 🚀 Python-3.11.9 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2060, 5919MiB)
(ClientAppActor pid=6270) engine/trainer: task=detect, mode=train, model=runs/detect/train5/weights/best.pt, data=datasets/roadsigns/config.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fa

train: Scanning /home/pierre/git/newEdge/datasets/roadsigns/labels/train.cache... 693 images, 0 backgrounds, 0 corrupt: 100%|██████████| 693/693 [00:00<?, ?it/s]
val: Scanning /home/pierre/git/newEdge/datasets/roadsigns/labels/val.cache... 184 images, 0 backgrounds, 0 corrupt: 100%|██████████| 184/184 [00:00<?, ?it/s]


(ClientAppActor pid=6270) Plotting labels to runs/detect/train10/labels.jpg... 
(ClientAppActor pid=6270) optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
(ClientAppActor pid=6270) optimizer: AdamW(lr=0.000476, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
(ClientAppActor pid=6270) Image sizes 640 train, 640 val
(ClientAppActor pid=6270) Using 8 dataloader workers
(ClientAppActor pid=6270) Logging results to runs/detect/train10
(ClientAppActor pid=6270) Starting training for 1 epochs...
(ClientAppActor pid=6270) 
(ClientAppActor pid=6270)       Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/44 [00:00<?, ?it/s]
        1/1      2.28G      0.643      1.743     0.8688         48        640:   2%|▏         | 1/44 [00:01<00:45,  1.05s/it]
        1/1      2.28G     0.7131      1.948     0.9201         53        640:   5%|▍         | 2/44 [00:01<00:23,  1.80it/s]
        1/1      2.28G     0.6728      1.852     0.9056         34        640:   7%|▋         | 3/44 [00:01<00:16,  2.51it/s]
        1/1      2.28G     0.7435      2.033     0.9152         65        640:   9%|▉         | 4/44 [00:01<00:12,  3.09it/s]
        1/1      2.31G     0.7004      2.007     0.9078         32        640:  11%|█▏        | 5/44 [00:01<00:11,  3.38it/s]
        1/1      2.31G      0.705      2.044     0.9008         49        640:  14%|█▎        | 6/44 [00:02<00:09,  4.07it/s]
        1/1      2.31G     0.7051      2.071     0.9008         42        640:  16%|█▌        | 7/44 [00:02<00:07,  4.63it/s]
        1/1      2.31G     0.7138      2.151     0.9161         43        640: 

(ClientAppActor pid=6270)                    all        184        254      0.997       0.38      0.804      0.671
(ClientAppActor pid=6270) 
(ClientAppActor pid=6270) 1 epochs completed in 0.003 hours.
(ClientAppActor pid=6270) Optimizer stripped from runs/detect/train10/weights/last.pt, 6.2MB
(ClientAppActor pid=6270) Optimizer stripped from runs/detect/train10/weights/best.pt, 6.2MB
(ClientAppActor pid=6270) 
(ClientAppActor pid=6270) Validating runs/detect/train10/weights/best.pt...
(ClientAppActor pid=6270) Ultralytics YOLOv8.2.32 🚀 Python-3.11.9 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2060, 5919MiB)
(ClientAppActor pid=6270) Model summary (fused): 168 layers, 3008963 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/6 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  17%|█▋        | 1/6 [00:00<00:00,  6.79it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  33%|███▎      | 2/6 [00:00<00:00,  6.00it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  50%|█████     | 3/6 [00:00<00:00,  3.46it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  67%|██████▋   | 4/6 [00:01<00:00,  2.70it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  83%|████████▎ | 5/6 [00:01<00:00,  3.62it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:01<00:00,  4.13it/s]


(ClientAppActor pid=6270)                    all        184        254      0.997       0.38      0.804      0.671
(ClientAppActor pid=6270)          traffic light         24         39          1      0.337      0.746      0.502
(ClientAppActor pid=6270)              stop sign         21         21      0.986      0.952      0.958      0.872
(ClientAppActor pid=6270)        speedlimit sign        140        161          1     0.0921      0.983      0.848
(ClientAppActor pid=6270)              crosswalk         31         33          1      0.139      0.527      0.462
(ClientAppActor pid=6270) Speed: 0.2ms preprocess, 2.0ms inference, 0.0ms loss, 2.6ms postprocess per image
(ClientAppActor pid=6270) Results saved to runs/detect/train10
(ClientAppActor pid=6270) Ultralytics YOLOv8.2.32 🚀 Python-3.11.9 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2060, 5919MiB)
(ClientAppActor pid=6270) engine/trainer: task=detect, mode=train, model=runs/detect/train5/weights/best.pt, data=datasets/roads

train: Scanning /home/pierre/git/newEdge/datasets/roadsigns/labels/train.cache... 693 images, 0 backgrounds, 0 corrupt: 100%|██████████| 693/693 [00:00<?, ?it/s]
val: Scanning /home/pierre/git/newEdge/datasets/roadsigns/labels/val.cache... 184 images, 0 backgrounds, 0 corrupt: 100%|██████████| 184/184 [00:00<?, ?it/s]


(ClientAppActor pid=6270) Plotting labels to runs/detect/train11/labels.jpg... 
